In [1]:
def deco(func):
  def inner():
    print('running inner()')
  return inner

In [3]:
@deco
def target():
  print('running target()')
target()

running inner()


In [4]:
target

<function __main__.deco.<locals>.inner()>

In [5]:
def logger(func):
  def wrapper(*args, **kwargs):
    print(f"[LOG] {func.__name__} called with args: {args}, kwargs: {kwargs}")
    result = func(*args, **kwargs)
    print(f"[LOG] {func.__name__} returned: {result}")
    return result
  return wrapper

@logger
def add(a,b):
  return a+b

In [8]:
registry = []

def register(func):
  print('running register(func)')
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()

running register(func)
running register(func)
running main()
registry -> [<function f1 at 0x7d824977c860>, <function f2 at 0x7d824977db20>]
running f1()
running f2()
running f3()


In [9]:
promos = []

def promotion(promo_func):
  """promotion() decoration는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
  promos.append(promo_func)
  return promo_func
@promotion
def fidelity_promo(order):
  """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
  return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
  """20개 이상의 동일 상품을 구매하면 10% 할인 적용"""
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
  """10종류 이상의 상품을 구매하면 전체 7% 할인 적용"""
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

def best_promo(order):
  """최대로 할인받을 금액을 반환한다"""
  return max(promo(order) for promo in promos)

In [10]:
def f1(a):
  print(a)
  print(b)

f1(3)

3


NameError: name 'b' is not defined

In [11]:
b = 6
def f2(a):
  print(a)
  print(b)
  b=9

f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [12]:
b= 6
def f3(a):
  global b
  print(a)
  print(b)
  b = 9
f3(3)

3
6


In [13]:
b

9

In [14]:
f3(3)

3
9


In [15]:
b=30
f3(3)

3
30


In [18]:
class Averager():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

In [19]:
avg = Averager()
avg(10)

10.0

In [20]:
avg(11)

10.5

In [21]:
avg(12)

11.0

In [22]:
def make_averager():
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)
  return averager

In [27]:
avg = make_averager()
avg(10)

10.0

In [28]:
avg(11)

10.5

In [29]:
avg(12)

11.0

In [30]:
avg.__code__.co_varnames

('new_value', 'total')

In [31]:
avg.__code__.co_freevars

('series',)

In [32]:
avg.__closure__

(<cell at 0x7d8249e8fe80: list object at 0x7d82487a0fc0>,)

In [33]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [34]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    count +=1
    total +=new_value
    return total/count

  return averager

In [35]:
avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [36]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count

  return averager

In [37]:
avg = make_averager()
avg(10)

10.0

In [38]:
import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() -t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [39]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12312837s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000060s] factorial(1) -> 1
[0.00001230s] factorial(2) -> 2
[0.00001792s] factorial(3) -> 6
[0.00002282s] factorial(4) -> 24
[0.00002846s] factorial(5) -> 120
[0.00003740s] factorial(6) -> 720
6! = 720


In [40]:
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
      arg_lst.append(','.join(pairs))
    arg_str = ','.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [41]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12312102s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000119s] factorial(1) -> 1
[0.00002384s] factorial(2) -> 2
[0.00003338s] factorial(3) -> 6
[0.00004435s] factorial(4) -> 24
[0.00005460s] factorial(5) -> 120
[0.00006819s] factorial(6) -> 720
6! = 720


In [42]:
factorial.__name__

'factorial'